In [ ]:
# CAENDigitizerControl - Written by Edgar Mao
#
# Adapted from demo_scope.py by Giovanni Cerretani

In [ ]:
import numpy as np

from caen_felib import device, error
import matplotlib.pyplot as plt


In [ ]:
# Digitizer settings
# TODO: Figure out if reclen, pre/post-trig are in units of samples or ns


# Board Configuration parameters
reclen = 2048             # record length, in units of ns
post_trigger = 1024       # ns
trig_edge = f"RISE"       # Self-trigger edge, "RISE" or "FALL"
stmode = f"START_MODE_SW" # Startmodes: "START_MODE_SW", "START_MODE_S_IN", or "START_MODE_FIRST_TRG"
dec_f = f"DECIM_FACTOR_1" # Decimation factor: "DECIM_FACTOR_1" and "DECIM_FACTOR_2^n" where n can be integers 1-7
ext_clock = f"FALSE"      # If set "TRUE" the digitizer will take external clock via CLK-IN
ext_trg = f"FALSE"        # If set "TRUE" the external trigger participates to the global trigger generation in logic OR with the other enabled signals
                          # External trigger signal needs to be a square wave with V_pp <= 1.5V




# group-specific settings
gr_on = [0,1,2,3]                # Group numbers to participate in the event readout
gr_off = []                      # Group numbers to NOT participate in the event readout (all groups participate by default)
gr_ntrig_gen = []                # Groups NOT participating in global trigger generation (all groups participate by default)
gr_trig_out = []                 # Groups participating in TRG-OUT at GPO (all groups DO NOT participate by default)
gr_trig_lvl = [1150,900,900,900] # in units of LSB (0-4095)
gr_dc = [i/40.96 for i in [840,840,840,840]] # in units of LSB (0-4095)



In [ ]:
# Variable for storing all acquired waveforms
allwf = []

# Save the data?
save = True
spath = f"\."

In [ ]:
# TODO [Features]:
    # Connectivity and live display [done]
    # Set trigger level [by group], bsline [by group], reclen [done], 
    # Save data [need revising]
    # Basic logic?
    # GUI? or some kind of UI
    # Runtime control
    # external trigger [done]
    # trigger output [by group]
# TODO: clarify settings and delete redudancy (ex. reclen)

In [ ]:
with device.connect('dig1://caen.internal/usb') as dig:
    # Reset
    dig.cmd.Reset()
    dig.cmd.ClearData()

    
    # Get board info
    nch = int(dig.par.NUMCH.value)
    n_analog_traces = int(dig.par.NUMANALOGTRACES.value)
    n_digital_traces = int(dig.par.NUMDIGITALTRACES.value)
    adc_samplrate_msps = float(dig.par.ADC_SAMPLRATE.value)  # in Msps
    adc_nbit = int(dig.par.ADC_NBIT.value)
    sampling_period_ns = int(1e3 / adc_samplrate_msps)
    fw_type = dig.par.FWTYPE.value


    # Configure digitizer
    dig.par.STARTMODE.value = stmode
    dig.par.RECLEN.value = f'{reclen}'
    dig.par.self_trigger_edge.value = trig_edge
    dig.par.posttrg.value = f'{post_trigger}'
    dig.par.decimation_factor.value = dec_f
    dig.par.dt_ext_clock.value = ext_clock
    dig.par.trg_ext_enable.value = ext_trg


    # Set group parameters
    for i in gr_off:
        dig.group[i].par.gr_enabled.value = f'FALSE'
    for i in gr_on:
        dig.group[i].par.gr_enabled.value = f'TRUE'
        dig.group[i].par.gr_threshold.value = f'{gr_trig_lvl[i]}'
        dig.group[i].par.gr_dcoffset.value = f'{gr_dc[i]}'
    for i in gr_ntrig_gen:
        dig.group[i].par.gr_trg_global_gen.value = f'FALSE'
    for i in gr_trig_out:
        dig.group[i].par.gr_out_propagate.value = f'TRUE'

    
    # Compute record length in samples
    reclen_ns = int(dig.par.RECLEN.value)  # Read back RECLEN to check if there have been rounding
    reclen = int(reclen_ns / sampling_period_ns)


    # Set enabled endpoint to activate decode
    dig.endpoint.par.ActiveEndpoint.value = 'scope'

    # Configure endpoint
    data_format = [
        {
            'name': 'TIMESTAMP',
            'type': 'U64',
        },
        {
            'name': 'WAVEFORM',
            'type': 'U16',
            'dim': 2,
            'shape': [nch, reclen],
        },
        {
            'name': 'WAVEFORM_SIZE',
            'type': 'U64',
            'dim': 1,
            'shape': [nch],
        },
    ]


    # Store endpoint node
    decoded_endpoint_path = fw_type.strip('-')  # decoded endpoint path is just firmware type without -
    endpoint = dig.endpoint[decoded_endpoint_path]
    # set_read_data_format returns allocated buffers
    data = endpoint.set_read_data_format(data_format)

    timestamp = data[0].value
    waveform = data[1].value
    waveform_size = data[2].value
    
    # setup live plot
    fig = plt.figure()
    hfig = display(fig, display_id=True)
    
    # Start acquisition
    dig.cmd.ArmAcquisition()

    # Acquisition loop
    def update():
        # TODO: Add a feature that controls the run time rather than just this for loop
        for _ in range(500):

            # Send software trigger
            #dig.cmd.SendSwTrigger()

            # Wait for event
            try:
                dig.endpoint.scope.read_data(0, data)
                allwf.append(waveform)
            except error.Error as ex:
                if ex.code == error.ErrorCode.TIMEOUT:
                    # Timeout expired, waiting again
                    continue
                elif ex.code == error.ErrorCode.STOP:
                    # End of run, exit the loop
                    print('End of run.')
                    break
                else:
                    # Other critical error, propagate it
                    raise ex

            for i in range(nch):
                plt.plot(np.arange(0, waveform_size[i]), waveform[i])

            # Change sampling rate of live plots
            '''
            if _%10:
                for i in range(nch):
                    plt.plot(np.arange(0, waveform_size[i]), waveform[i])
            '''

            fig.canvas.draw()
            hfig.update(fig)
            fig.canvas.flush_events()
            plt.clf()
            
        
    update()

    # Stop acquisition
    dig.cmd.DisarmAcquisition()

In [ ]:
# Save data
# Each triggered event is saved as a separate file

for i in range(len(allwf)):
    np.savetxt(f"wf{i}.csv",
            allwf[i],
            delimiter =", ",
            fmt ='% s')

In [ ]:
# Note: right now the channels are triggering normally but the buffer is being filled up, for low rate studies (<10Hz) the current live plot feature is fine
# for high rate use cases need to lower the sampling rate of the live plot feature
# TODO: high rate feature, complete setting file, releasable version, trig in